In [1]:
from pathlib import Path
from tqdm import tqdm
import requests
import csv

# Better than pandas
import polars as pl

In [16]:
pl.Config.set_tbl_rows(100)
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [2]:
countries = (
    pl.scan_csv("./landlocked 2023-06-26 - Лист1.csv")
    .select(pl.col("landlocked country").alias("name"))
    .collect()
)
countries

name
str
"""Switzerland"""
"""Austria"""
"""Czech Republic…"
"""Hungary"""
"""Serbia"""
"""Slovakia"""
"""Ethiopia"""
"""Kazakhstan"""
"""Luxembourg"""


In [3]:
def wbsearchentities(name: str):
    response = requests.get(
        "https://www.wikidata.org/w/api.php",
        params={
            "action": "wbsearchentities",
            "search": name,
            "language": "en",
            "format": "json",
        },
    )
    response.raise_for_status()
    return response.json()

In [4]:
search_results_by_name = {}
for name in tqdm(countries["name"], total=len(countries)):
    search_results_by_name[name] = wbsearchentities(name)

100%|█████████████████████████████████████████████████████████████████████████████████| 44/44 [00:17<00:00,  2.57it/s]


In [17]:
def min_description(search_result: dict):
    return {
        "code": search_result["id"],
        "label": search_result["label"],
        "description": search_result["description"],
    }


countries.with_columns(
    pl.col("name")
    .apply(lambda name: min_description(search_results_by_name[name]["search"][0]))
    .alias("found")
).unnest("found")

name,code,description,label
str,str,str,str
"""Switzerland""","""Q39""","""country in Central Europe""","""Switzerland"""
"""Austria""","""Q40""","""country in Central Europe""","""Austria"""
"""Czech Republic""","""Q213""","""country in Central Europe""","""Czech Republic"""
"""Hungary""","""Q28""","""country in Central Europe""","""Hungary"""
"""Serbia""","""Q403""","""country in southeastern Europe""","""Serbia"""
"""Slovakia""","""Q214""","""country in Central Europe""","""Slovakia"""
"""Ethiopia""","""Q115""","""country in the Horn of Africa""","""Ethiopia"""
"""Kazakhstan""","""Q232""","""sovereign state in Eastern Europe and Central Asia""","""Kazakhstan"""
"""Luxembourg""","""Q32""","""country in Western Europe""","""Luxembourg"""
